## Notebook to process GNSS data for each receiver

### Step 1: Load libraries

Chunk that sloads necessary packages & sets working environment to where the jupyterlab notebook file is 

In [1]:
%load_ext autoreload
%autoreload 2
import gnssvod as gv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pdb
import importlib
import zipfile
import os
import xarray as xr
import glob
import shutil
from gnssvod.hemistats.hemistats import hemibuild
#import georinex as gr
#import qgrid as interactive table 
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates
import shutil
from matplotlib import rcParams
from datetime import datetime, timedelta

### Step 2: Time intervals

Set up time intervals for the study period. The argument **periods** represents the number of days from the start day.

In [2]:
startday = pd.to_datetime('01-03-2025',format='%d-%m-%Y')
timeintervals=pd.interval_range(start=startday, periods=10, freq='D', closed='left')
timeintervals

IntervalIndex([[2025-03-01 00:00:00, 2025-03-02 00:00:00),
               [2025-03-02 00:00:00, 2025-03-03 00:00:00),
               [2025-03-03 00:00:00, 2025-03-04 00:00:00),
               [2025-03-04 00:00:00, 2025-03-05 00:00:00),
               [2025-03-05 00:00:00, 2025-03-06 00:00:00),
               [2025-03-06 00:00:00, 2025-03-07 00:00:00),
               [2025-03-07 00:00:00, 2025-03-08 00:00:00),
               [2025-03-08 00:00:00, 2025-03-09 00:00:00),
               [2025-03-09 00:00:00, 2025-03-10 00:00:00),
               [2025-03-10 00:00:00, 2025-03-11 00:00:00)],
              dtype='interval[datetime64[ns], left]')

### Step 3: Read RINEX file

We then proceed to read RINEX file. This recquires to indicate the location of the file (**pattern**) and the location of the output directory (**outputdir**). Sometimes we could get the following error:

ValueError: Missing an approximate antenna position. Provide the argument ‘approx_position’ to preprocess()

In [ ]:
pattern= {'Reach-2':'/Volumes/coraldelmar/Reach-02_raw_20250322184422.25O'}
outputdir={'Reach-2':'/Volumes/coraldelmar/nc/'}
#approx_position=[-4705.036,43.000,23011766.990]
keepvars=['S1C','S1X','S2C','S2X']
#gv.preprocess(pattern,interval='30s',keepvars=keepvars,outputdir=outputdir, approx_position=approx_position)
gv.preprocess(pattern,interval='30s',keepvars=keepvars,outputdir=outputdir)